# Medical Examiner Case Archives

Cook County (Chicago) medical examiner records, taken from [here](https://datacatalog.cookcountyil.gov/Public-Safety/Medical-Examiner-Case-Archive/cjeq-bs86) after discovery via [Data is Plural](https://tinyletter.com/data-is-plural).

## Do your importing/setup

In [1]:
import pandas as pd

%matplotlib inline

## Read in the data, check its row count and column types

In [2]:
!ls

Medical Examiner.ipynb            case-archive-encoded.csv
Medical_Examiner_Case_Archive.csv


In [3]:
df = pd.read_csv('case-archive-encoded.csv', na_values=['-1', 'NaN'])

In [4]:
df = df.rename(columns={
    'Case Number': 'Case_Number',
    'Date of Incident': 'Date_of_Incident',
    'Date of Death': 'Date_of_Death',
    'Manner of Death': 'Manner_of_Death',
    'Primary Cause': 'Primary_Cause',
    'Primary Cause Line B': 'Primary_Cause_Line_B',
    'Primary Cause Line C': 'Primary_Cause_Line_C',
    'Secondary Cause': 'Secondary_Cause',
    'Gun Related': 'Gun_Related',
    'Opioid Related': 'Opioid_Related',
    'Incident Address': 'Incident_Address',
    'Incident City': 'Incident_City'
})
df.tail()

,Case_Number,Date_of_Incident,Date_of_Death,Age,Gender,Race,Latino,Manner_of_Death,Primary_Cause,Primary_Cause_Line_B,Primary_Cause_Line_C,Secondary_Cause,Gun_Related,Opioid_Related,Incident_Address,Incident_City,longitude,latitude,OBJECTID,location
22629,ME2018-02104,12/31/2017 11:23:00 PM,05/02/2018 01:59:00 PM,34.0,1.0,5.0,False,PENDING,PENDING,NaN,NaN,NaN,NaN,NaN,5535 NORTH LINCOLN AVENUE,CHICAGO,-87.693476,41.982498,78129,"(41.9824976, -87.6934761)"
22630,ME2018-02122,05/03/2018 12:29:00 PM,05/03/2018 12:39:00 PM,94.0,1.0,5.0,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,"CORONARY ARTERY DISEASE, HYPERTENSION, DIABETE...",NaN,NaN,16416 EVANS AVENUE,SOUTH HOLLAND,-87.600994,41.596634,78272,"(41.5966344, -87.6009945)"
22631,ME2018-02114,05/03/2018 08:09:00 AM,05/03/2018 08:19:00 AM,68.0,1.0,0.0,False,PENDING,PENDING,NaN,NaN,NaN,NaN,NaN,15243 3RD AVE.,PHOENIX,-87.631849,41.611941,78496,"(41.6119409, -87.6318491)"
22632,ME2018-02105,05/02/2018 02:00:00 PM,05/02/2018 02:25:00 PM,50.0,1.0,5.0,False,SUICIDE,GUNSHOT WOUND OF HEAD,NaN,NaN,NaN,True,NaN,640 CUMBERLAND ST,HOFFMAN ESTATES,-88.079340,42.039490,78745,"(42.0394902, -88.07934)"
22633,ME2018-02107,05/02/2018 04:53:00 PM,05/02/2018 05:11:00 PM,57.0,1.0,2.0,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,NaN,NaN,3914 212 PL,MATTESON,-87.711457,41.504114,78828,"(41.5041136, -87.7114567)"


## Cleaning up your data

First you'll want to convert the `Race` and `Gender` columns from codes into actual text to make analysis easier.

### Gender codes

* `-1` - `Data missing`
* `0` - `Female`
* `1` - `Male`
* `2` - `Unknown`

### Race codes

* `-1` - `Data missing`
* `0` - `American Indian`
* `1` - `Asian`
* `2` - `Black`
* `3` - `Other`
* `4` - `Unknown`
* `5` - `White`

In [5]:
df['Gender'] = df.Gender.replace({
    -1: 'Data missing',
    0: 'Female',
    1: 'Male',
    2: 'Unknown'
})

In [6]:
df['Race'] = df.Race.replace({
    -1: 'Data missing',
    0: 'American Indian',
    1: 'Asian',
    2: 'Black',
    3: 'Other',
    4: 'Unknown',
    5: 'White'
})

## What percent of the dataset is men, and what percent is women?

It should display as **Male** and **Female**, not as numbers.

In [7]:
df.Gender.value_counts(normalize=True) * 100

Male       71.708161
Female     28.238583
Unknown     0.053255
Name: Gender, dtype: float64

## Getting rid of "Data missing"

`Unknown` means that officially the gender or race is unknown, while `Data missing` means the record is incomplete. That means "Data missing" should have been `NaN`!

Go back to your `read_csv` many cells before and make it so that "Data missing" is automatically set as `NaN`.

- *Tip: Do not use `.replace` for this one!*
- *Tip: Look at the options for `read_csv`, there's something that lets you specify missing values*
- *Tip: It isn't `"Data missing"` - think about how you already replaced*
- *Tip: Be sure you're using an array when you tell it what the 'missing' options are*

### After you've done this, re-run all of the the previous cells and confirm that `"Data missing"` does not exist any more

## What is the most common race in the dataset? We want percentages.

We'll come back to this later, I'm just having you check the column for now.

In [8]:
df.Race.value_counts(normalize=True) * 100

White              55.486450
Black              41.608174
Asian               1.519325
Other               1.195024
Unknown             0.124389
American Indian     0.066637
Name: Race, dtype: float64

## Do a `.value_counts()` on the `Opioid Related` column

In [9]:
df.Opioid_Related.astype(float).value_counts()

1.0    3378
Name: Opioid_Related, dtype: int64

## That's weird. Did everyone die from opioids? Try again, but including missing data.

In [10]:
df.Opioid_Related.astype(float).value_counts(dropna=False)

NaN    19256
1.0     3378
Name: Opioid_Related, dtype: int64

## Cleaning up True/False columns

For some reason in this dataset, the True/False columns are either `True` or `NaN`. `NaN` causes a lot of problems, I'd rather have it be false.

You can use [`fillna`](http://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.fillna.html) to fill in empty data - it's like `.replace` but for `NaN`.

### Replace all `NaN` values with `False` for the "Gun Related" and "Opioid Related" columns.

In [11]:
df.Opioid_Related.fillna(value=False)

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15        True
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
22604    False
22605    False
22606    False
22607    False
22608    False
22609    False
22610    False
22611    False
22612    False
22613    False
22614    False
22615    False
22616    False
22617    False
22618    False
22619    False
22620    False
22621    False
22622    False
22623    False
22624    False
22625    False
22626    False
22627    False
22628    False
22629    False
22630    False
22631    False
22632    False
22633    False
Name: Opioid_Related, Length: 22634, dtype: bool

In [12]:
df.Gun_Related.fillna(False)

0        False
1        False
2        False
3        False
4        False
5        False
6         True
7        False
8        False
9        False
10        True
11       False
12       False
13       False
14        True
15       False
16       False
17       False
18       False
19        True
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
22604    False
22605    False
22606    False
22607    False
22608    False
22609    False
22610    False
22611    False
22612    False
22613    False
22614    False
22615    False
22616    False
22617    False
22618    False
22619    False
22620    False
22621    False
22622    False
22623    False
22624    False
22625    False
22626    False
22627    False
22628     True
22629    False
22630    False
22631    False
22632     True
22633    False
Name: Gun_Related, Length: 22634, dtype: bool

### Do another value counts on Opioid Related to make sure it has both True and False values

In [13]:
df.Opioid_Related.isna()

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15       False
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
         ...  
22604     True
22605     True
22606     True
22607     True
22608     True
22609     True
22610     True
22611     True
22612     True
22613     True
22614     True
22615     True
22616     True
22617     True
22618     True
22619     True
22620     True
22621     True
22622     True
22623     True
22624     True
22625     True
22626     True
22627     True
22628     True
22629     True
22630     True
22631     True
22632     True
22633     True
Name: Opioid_Related, Length: 22634, dtype: bool

# Back to analysis!

## What's the average age people were when they died?

In [14]:
df.Age.mean()

51.279928793947484

## Let's look at how the oldest people died

We're just going to browse. Read through how the **oldest 30 people died.**

In [15]:
df[['Primary_Cause', 'Age']].sort_values(by='Age', ascending = False).head(30)

,Primary_Cause,Age
21142,COMPLICATIONS OF A FALL,106.0
17602,ACUTE MYOCARDIAL INFARCTION,106.0
12106,ORGANIC CARDIOVASCULAR DISEASE,105.0
4942,COMPLICATIONS OF FALL,103.0
9376,ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE,103.0
8653,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,103.0
14697,INHALATION OF PRODUCTS OF COMBUSTION,101.0
11558,COMPLICATIONS OF LEFT FEMORAL NECK FRACTURE,101.0
9904,ORGANIC CARDIOVASCULAR DISEASE,101.0
13498,ORGANIC CARDIOVASCULAR DISEASE,101.0


## Seems like a lot of problems with fractures

### What's the median age of someone dying from a cause that involves a fracture?

Are fractures especially dangerous for the elderly?

- *Tip: Filter for a cause that involves a fracture, then take the median age*
- *Tip: If you get a "cannot index NA values" error, the problem is it's trying to search `NaN` values and doesn't know what to do with them. You need to tell pandas to count `NaN` as false by setting another option - it isn't `NaN=False`, but it's close!*

In [16]:
df[df.Primary_Cause.str.contains("FRACTURE", na=False)]

,Case_Number,Date_of_Incident,Date_of_Death,Age,Gender,Race,Latino,Manner_of_Death,Primary_Cause,Primary_Cause_Line_B,Primary_Cause_Line_C,Secondary_Cause,Gun_Related,Opioid_Related,Incident_Address,Incident_City,longitude,latitude,OBJECTID,location
173,ME2018-02232,05/10/2018 05:15:00 AM,05/10/2018 05:27:00 AM,98.0,Male,White,False,ACCIDENT,COMPLICATIONS OF FEMUR FRACTURE,NaN,NaN,HYPERTENSIVE CARDIOVASCULAR DISEASE,NaN,NaN,55 E. PEARSON ST.,CHICAGO,-87.626284,41.897293,78038,"(41.8972932, -87.6262844)"
325,ME2018-01200,02/27/2018 09:57:00 AM,03/07/2018 06:32:00 AM,86.0,Male,White,False,ACCIDENT,COMPLICATIONS OF FRACTURE OF HIP,NaN,NaN,HYPERTENSIVE-ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,4101 LAKE COOK RD,NORTHBROOK,-87.879300,42.151985,60909,"(42.1519851, -87.8792996)"
742,ME2018-01660,02/17/2018 12:00:00 AM,03/25/2018 07:05:00 AM,84.0,Female,White,False,ACCIDENT,LEFT FEMORAL NECK FRACTURE,NaN,NaN,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,2150 W. GOLF ROAD - ASSITED LIVING,HOFFMAN ESTATES,-88.132521,42.047745,60994,"(42.0477454, -88.1325211)"
886,ME2018-01684,03/30/2018 11:08:00 PM,04/06/2018 08:50:00 PM,85.0,Female,White,False,ACCIDENT,COMPLICATIONS OF PELVIS FRACTURES,NaN,NaN,"MALIGNANCY, ATRIAL FIBRILLATION, REMOTE CEREBR...",NaN,NaN,920 E. NORTHWEST HIGHWAY,PALATINE,-88.020844,42.101709,61503,"(42.101709, -88.0208439)"
911,ME2017-05372,11/15/2017 06:25:00 PM,11/15/2017 07:10:00 PM,84.0,Male,Asian,False,ACCIDENT,COMPLICATIONS OF RIGHT HIP FRACTURE,NaN,NaN,ALZHEIMER?S DISEASE AND ARTERIOSCLEROTIC CARDI...,NaN,NaN,504 W WELLINGTON ST,CHICAGO,-87.642031,41.936614,75882,"(41.936614, -87.6420313)"
951,ME2018-01756,04/05/2018 10:00:00 PM,04/11/2018 12:22:00 AM,92.0,Female,White,False,ACCIDENT,FRACTURE OF HIP,NaN,NaN,HYPERTENSIVE-ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,UNKNKOWN,NaN,NaN,NaN,55654,NaN
955,ME2018-01763,03/29/2018 12:00:00 AM,04/08/2018 07:20:00 AM,88.0,Female,White,False,ACCIDENT,RIGHT FEMUR NECK FRACTURE,NaN,NaN,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,480 N WOLF RD,NORTHLAKE,-87.904584,41.916897,72548,"(41.9168967, -87.9045839)"
1087,ME2018-01845,03/29/2018 08:00:00 AM,04/16/2018 01:45:00 AM,97.0,Female,White,False,ACCIDENT,COMPLICATIONS OF LEFT HIP FRACTURE,NaN,NaN,HYPERTENSIVE AND VALVULAR CARDIOVASCULAR DISEA...,NaN,NaN,"Sunrise NH, 1601 Green Bay Road",HIGHLAND PARK,NaN,NaN,57634,NaN
1128,ME2018-01879,01/30/2017 12:00:00 AM,01/31/2017 03:51:00 AM,93.0,Female,White,False,ACCIDENT,COMPLICATIONS OF LEFT FEMUR FRACTURE,NaN,NaN,"CORONARY ARTERY DISEASE, CHRONIC OBSTRUCTIVE P...",NaN,NaN,701 W. North Avenue,MELROSE PARK,-87.842431,41.910330,61835,"(41.91033, -87.8424306)"
1241,ME2014-00114,08/03/2014 05:53:00 PM,08/16/2014 07:37:00 PM,94.0,Male,White,False,ACCIDENT,COMPLICATIONS OF HIP FRACTURE,NaN,NaN,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,401 w lake st,NORTHLAKE,-87.917790,41.905397,58345,"(41.9053973, -87.9177899)"


In [17]:
df[df.Primary_Cause.str.contains("FRACTURE", na=False)].Age.median()

86.0

### To get a "compared to what?", what's the median age of _anyone_ dying an accidental death?

In [18]:
df[df.Manner_of_Death.str.contains("ACCIDENT", na=False)].Age.median()

51.0

### What's the median age of each manner of death?

It looks like different kinds of death might happen to different ages of people. Let's investigate that further.

In [19]:
df.groupby(by='Manner_of_Death').Age.median()

Manner_of_Death
ACCIDENT        51.0
HOMICIDE        26.0
NATURAL         61.0
PENDING         47.0
SUICIDE         45.0
UNDETERMINED    21.0
Name: Age, dtype: float64

### Who is the oldest homicide victim?

It looks like homicide is for young people, so maybe we'll find an interesting outlier?

In [20]:
df[df.Manner_of_Death.str.contains("HOMICIDE", na=False)].Age.max()

98.0

## Investigating toxicity-related homicides

She was old, and was purposefully overdosed on morphine and hydrocodone. Might have been euthenasia? Let's find similar cases.

### Find every homicide where the primary cause of death is some sort of toxicity

Toxicity can just overdose. You should have **ten rows**.

- *Tip: If you're doing this as one statement, make sure you use your parentheses correctly. If you leave them out, you'll have zero rows*
- *Tip: You could make a homicides-only dataframe if you wanted to*

In [21]:
df[(df.Primary_Cause.str.contains("TOXICITY", na=False)) & (df.Manner_of_Death.str.contains("HOMICIDE", na=False))]

,Case_Number,Date_of_Incident,Date_of_Death,Age,Gender,Race,Latino,Manner_of_Death,Primary_Cause,Primary_Cause_Line_B,Primary_Cause_Line_C,Secondary_Cause,Gun_Related,Opioid_Related,Incident_Address,Incident_City,longitude,latitude,OBJECTID,location
3702,ME2015-00519,02/03/2015 08:21:00 AM,02/04/2015 08:34:00 AM,98.0,Female,Black,False,HOMICIDE,MORPHINE AND HYDROCODONE TOXICITY,NaN,NaN,HYPERTENSIVE AND ATHEROSCLEROTIC CARDIOVASCULA...,NaN,True,16300 louis Ave,SOUTH HOLLAND,-87.609278,41.598109,64404,"(41.5981086, -87.6092784)"
5395,ME2015-02286,05/27/2015 03:15:00 PM,06/03/2015 07:25:00 PM,28.0,Female,White,False,HOMICIDE,"COMPLICATIONS OF AMITRIPTYLINE AND METABOLITE,...",NaN,NaN,NaN,NaN,NaN,1102 S. Springinsguth Rd. #1D,SCHAUMBURG,-88.119741,42.005376,60663,"(42.005376, -88.1197414)"
6144,ME2015-03058,07/18/2015 07:00:00 AM,07/19/2015 02:05:00 AM,10.0,Male,Black,False,HOMICIDE,CARBON MONOXIDE TOXICITY,INCENDIARY HOUSE FIRE,NaN,NaN,NaN,NaN,8327 S. DREXEL,CHICAGO,-87.602328,41.743167,68218,"(41.7431675, -87.6023283)"
6764,ME2015-03692,08/29/2015 09:41:00 AM,08/29/2015 01:00:00 PM,28.0,Male,White,True,HOMICIDE,CARBON MONOXIDE TOXICITY,HOUSE FIRE,NaN,NaN,NaN,NaN,5209 S Lorel Avenue,CHICAGO,-87.755913,41.798035,72663,"(41.7980351, -87.7559129)"
6765,ME2015-03693,08/29/2015 09:41:00 AM,08/29/2015 01:00:00 PM,63.0,Male,Other,False,HOMICIDE,CARBON MONOXIDE TOXICITY,HOUSE FIRE,NaN,"ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE, ACUTE...",NaN,NaN,5205 S. Lorel,CHICAGO,-87.756075,41.798102,73036,"(41.7981017, -87.7560751)"
7612,ME2015-04564,10/22/2015 10:39:00 AM,10/22/2015 11:05:00 AM,31.0,Male,Other,False,HOMICIDE,HEROIN AND ETHANOL TOXICITY,NaN,NaN,NaN,NaN,True,5240 NORTH SHERIDAN APT 417,CHICAGO,-87.655620,41.977582,74058,"(41.977582, -87.6556196)"
12656,ME2016-04092,08/23/2016 01:37:00 AM,08/23/2016 06:45:00 AM,4.0,Female,Black,False,HOMICIDE,THERMAL INJURIES AND CARBON MONOXIDE TOXICITY,NaN,NaN,NaN,NaN,NaN,8112 S. Essex Avenue,CHICAGO,-87.564245,41.747711,68043,"(41.7477108, -87.5642452)"
12657,ME2016-04093,08/23/2016 01:37:00 AM,08/23/2016 06:45:00 AM,7.0,Female,Black,False,HOMICIDE,THERMAL INJURIES AND CARBON MONOXIDE TOXICITY,NaN,NaN,NaN,NaN,NaN,8112 S. Essex Avenue,CHICAGO,-87.564245,41.747711,68054,"(41.7477108, -87.5642452)"
12660,ME2016-04096,08/23/2016 01:37:00 AM,08/23/2016 06:45:00 AM,56.0,Male,Black,False,HOMICIDE,THERMAL INJURIES AND CARBON MONOXIDE TOXICITY,NaN,NaN,NaN,NaN,NaN,8114 S. Essex Avenue,CHICAGO,-87.564246,41.747673,67699,"(41.7476732, -87.5642456)"
13050,ME2016-04490,09/16/2016 02:45:00 AM,09/16/2016 03:34:00 AM,59.0,Male,White,True,HOMICIDE,CARBON MONOXIDE TOXICITY,GARAGE FIRE,NaN,NaN,NaN,NaN,2151 W 21st Street,CHICAGO,-87.680361,41.853700,65250,"(41.8536999, -87.6803607)"


### Okay, nope, we were wrong.

Those were almost **all from fires**. Apparently homicide is not the best place to go looking for toxicity. What's the most popular manner of death for primary causes involving toxicity?

- *Tip: Remember that `['colname']` is the same as `.colname`. You can't do `.col with spaces` so you'll need to do `['col with spaces']` a lot in this dataset
- *Tip: Or I guess if you really wanted to, you could rename your columns to have spaces in them (IF YOU DO THIS DON'T DO IT IN EXCEL BECAUSE IT WILL PROBABLY BREAK YOUR CSV.)*

In [22]:
df[df.Primary_Cause.str.contains("TOXICITY", na=False)].groupby(by='Manner_of_Death').Manner_of_Death.count()

Manner_of_Death
ACCIDENT        3236
HOMICIDE          10
NATURAL            1
SUICIDE          221
UNDETERMINED      42
Name: Manner_of_Death, dtype: int64

### Okay, toxicity deaths (overdoses) are mostly accidents. Let's look at the first 30 accidental deaths involving toxicity.

- *Tip: Remember your parentheses!*

In [23]:
df[(df.Primary_Cause.str.contains("TOXICITY", na=False)) & (df.Manner_of_Death.str.contains("ACCIDENT", na=False))].head(30)

,Case_Number,Date_of_Incident,Date_of_Death,Age,Gender,Race,Latino,Manner_of_Death,Primary_Cause,Primary_Cause_Line_B,Primary_Cause_Line_C,Secondary_Cause,Gun_Related,Opioid_Related,Incident_Address,Incident_City,longitude,latitude,OBJECTID,location
15,ME2018-01127,03/03/2018 02:15:00 AM,03/03/2018 02:24:00 AM,37.0,Male,White,False,ACCIDENT,"COMBINED DRUG (COCAINE, HEROIN, FENTANYL) TOXI...",NaN,NaN,NaN,NaN,True,6547 W. 111TH APT W-27,WORTH,-87.781732,41.689640,63784,"(41.6896403, -87.7817317)"
45,ME2018-00910,02/19/2018 08:00:00 PM,02/19/2018 08:22:00 PM,56.0,Male,Black,False,ACCIDENT,COMBINED DRUG (COCAINE AND HEROIN) TOXICITY,NaN,NaN,NaN,NaN,True,7724 SOUTH LOOMIS BLVD #1,CHICAGO,-87.658932,41.753282,69589,"(41.7532818, -87.6589318)"
51,ME2018-00400,01/20/2018 11:15:00 AM,01/20/2018 11:57:00 AM,19.0,Male,White,False,ACCIDENT,HEROIN TOXICITY,NaN,NaN,NaN,NaN,True,"5719 S. Kimbark, Apt. 3",CHICAGO,-87.594142,41.790743,77880,"(41.7907435, -87.5941419)"
53,ME2018-00643,02/03/2018 10:45:00 AM,02/03/2018 11:11:00 AM,35.0,Male,White,False,ACCIDENT,"ETHANOL AND COMBINED DRUG (PROBABLE HEROIN, CY...",NaN,NaN,NaN,NaN,True,"2540 N. Manheim Road, Rm#215",FRANKLIN PARK,NaN,NaN,77879,NaN
54,ME2018-00671,02/05/2018 12:00:00 AM,02/05/2018 12:10:00 AM,27.0,Male,White,False,ACCIDENT,COMBINED FENTANYL AND PROBABLE HEROIN TOXICITY,NaN,NaN,OBESITY,NaN,True,3435 W. VAN BUREN,CHICAGO,-87.711994,41.875391,77910,"(41.8753913, -87.7119935)"
65,ME2018-01129,03/03/2018 03:45:00 AM,03/03/2018 04:31:00 AM,34.0,Male,Black,False,ACCIDENT,ACUTE ETHANOL TOXICITY,NaN,NaN,NaN,NaN,NaN,7899 SOUTH SHORE,CHICAGO,-87.548113,41.752056,68080,"(41.7520559, -87.5481134)"
69,ME2018-01356,03/15/2018 01:31:00 PM,03/15/2018 01:44:00 PM,26.0,Male,White,False,ACCIDENT,"FENTANYL, HEROIN, AND ALPRAZOLAM TOXICITY",NaN,NaN,NaN,NaN,True,710 N. 4TH AVENUE,MAYWOOD,-87.838564,41.893982,77870,"(41.8939818, -87.8385636)"
93,ME2018-01560,03/29/2018 06:50:00 PM,03/29/2018 07:12:00 PM,38.0,Male,Asian,False,ACCIDENT,COCAINE AND FENTANYL TOXICITY,NaN,NaN,NaN,NaN,True,535 N MICHIGAN AVE #2705,CHICAGO,-87.623541,41.891975,66297,"(41.8919752, -87.6235405)"
99,ME2018-01570,03/30/2018 03:33:00 PM,03/30/2018 04:20:00 PM,50.0,Male,White,False,ACCIDENT,"COCAINE, FENTANYL AND ACETYL FENTANYL TOXICITY",NaN,NaN,NaN,NaN,True,1042 N. KNOLLWOOD DR. UNIT A,SCHAUMBURG,-88.138219,42.044579,62417,"(42.0445787, -88.1382195)"
102,ME2018-01574,03/30/2018 10:43:00 PM,03/30/2018 11:03:00 PM,40.0,Female,White,False,ACCIDENT,"COMBINED DRUG (FENTANYL, METHADONE, AND ALPRAZ...",NaN,NaN,NaN,NaN,True,4345 AUTHUR AVE.,BROOKFIELD,NaN,NaN,63618,NaN


## Wow, that's a lot of drug overdoses. What's more popular for overdosing: heroin, fentanyl, cocaine, or ethanol?

You can count something like "COMBINED ETHANOL, NORDIAZEPAM, AND FENTANYL TOXICITY" under both ethanol and fentanyl.

- *Tip: Search for them individually*

In [24]:
df[(df.Primary_Cause.str.contains("TOXICITY" and "HEROIN", na=False)) & (df.Manner_of_Death.str.contains("ACCIDENT", na=False))].Primary_Cause.count()


2009

In [25]:
df[(df.Primary_Cause.str.contains("TOXICITY" and "FENTANYL", na=False)) & (df.Manner_of_Death.str.contains("ACCIDENT", na=False))].Primary_Cause.count()


1484

In [26]:
df[(df.Primary_Cause.str.contains("TOXICITY" and "COCAINE", na=False)) & (df.Manner_of_Death.str.contains("ACCIDENT", na=False))].Primary_Cause.count()


1059

In [27]:
df[(df.Primary_Cause.str.contains("TOXICITY" and "ETHANOL", na=False)) & (df.Manner_of_Death.str.contains("ACCIDENT", na=False))].Primary_Cause.count()


909

# Cleaning up Primary Cause

Let's stop investigating for a second and maybe clean up this "Primary Cause" column.

## What are the most common Primary Cause of death? Include `NaN` values

- *Tip: There is an option that keeps `NaN` values when counting things in a column.*

In [28]:
df.groupby(by='Primary_Cause').Primary_Cause.count().sort_values(ascending=False).head(10)

Primary_Cause
ORGANIC CARDIOVASCULAR DISEASE                              1983
MULTIPLE GUNSHOT WOUNDS                                     1377
HYPERTENSIVE CARDIOVASCULAR DISEASE                         1177
ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE                      974
HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE         960
MULTIPLE INJURIES                                            941
HYPERTENSIVE AND ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE     460
PENDING                                                      452
GUNSHOT WOUND OF HEAD                                        440
HANGING                                                      323
Name: Primary_Cause, dtype: int64

## That was horrible looking. I don't want to read through that - how many `NaN` causes of death are there?

- *Tip: You can use `isnull()` to see if it's missing data, but how do you count the results?*

In [29]:
df.Primary_Cause.isna().sum()

124

## Remove all rows where the primary cause of death has not been filled out.

- *Tip: confirm that you have 22510 rows when you're done*

In [30]:
df = df.dropna(subset=['Primary_Cause'])

In [31]:
df.shape

(22510, 20)

# Cardiovascular disease

Cardiovascular disease (heart disease) is the number one or number two killer in America.

### Filter for only rows where cardiovascular disease was a primary cause

- *Tip: I hope you know how to deal with the `NaN` error message by now!*

In [32]:
df[(df.Primary_Cause.str.contains("CARDIOVASCULAR DISEASE", na=False))]

,Case_Number,Date_of_Incident,Date_of_Death,Age,Gender,Race,Latino,Manner_of_Death,Primary_Cause,Primary_Cause_Line_B,Primary_Cause_Line_C,Secondary_Cause,Gun_Related,Opioid_Related,Incident_Address,Incident_City,longitude,latitude,OBJECTID,location
18,ME2018-01135,03/03/2018 09:30:00 AM,03/03/2018 10:05:00 AM,55.0,Male,Black,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,NaN,NaN,NaN,10034 S. MICHICAGN AVE. APT 2,NaN,NaN,NaN,71064,NaN
27,ME2018-02131,05/03/2018 07:15:00 PM,05/03/2018 08:00:00 PM,48.0,Male,White,False,NATURAL,ATHEROSCLEROTIC AND HYPERTENSIVE CARDIOVASCULA...,NaN,NaN,OBESITY,NaN,NaN,2631 West Rascher Avenue,CHICAGO,-87.695479,41.980215,77843,"(41.9802154, -87.6954789)"
34,ME2018-02170,05/06/2018 02:05:00 PM,05/06/2018 02:46:00 PM,64.0,Male,Black,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,NaN,NaN,NaN,4250 S. PRINCETON APT 412,CHICAGO,-87.634704,41.816742,78016,"(41.8167422, -87.6347043)"
43,ME2014-01078,10/18/2014 02:30:00 PM,10/18/2014 03:55:00 PM,67.0,Male,White,False,NATURAL,ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE,NaN,NaN,NaN,NaN,NaN,610 SOUTH MAWATA,MOUNT PROSPECT,NaN,NaN,61280,NaN
57,ME2018-02186,05/07/2018 12:26:00 PM,05/07/2018 12:53:00 PM,66.0,Male,Black,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,NaN,NaN,NaN,146 N. CENTRAL #1D,CHICAGO,-87.765474,41.883634,78010,"(41.8836343, -87.7654736)"
64,ME2018-00995,02/23/2018 09:35:00 AM,02/23/2018 09:50:00 AM,70.0,Male,Black,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,NaN,NaN,NaN,5700 NORTH SHERIDAN APT 1003,NaN,-87.655755,41.985857,55722,"(41.9858569, -87.6557545)"
66,ME2018-01148,03/03/2018 04:30:00 PM,03/03/2018 05:09:00 PM,58.0,Female,White,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,OBESITY,NaN,NaN,11011 JORDAN DRIVE,OAK LAWN,NaN,NaN,59051,NaN
71,ME2018-01411,03/19/2018 12:00:00 AM,03/19/2018 11:04:00 AM,59.0,Female,Black,False,NATURAL,HYPERTENSIVE CARDIOVASCULAR DISEASE,NaN,NaN,"CHRONIC OBSTRUCTIVE PULMONARY DISEASE, DIABETE...",NaN,NaN,7804 South Maryland 1st floor,CHICAGO,-87.604086,41.753007,77882,"(41.7530066, -87.6040862)"
72,ME2018-01910,05/01/2018 10:50:00 AM,05/01/2018 10:59:00 AM,65.0,Male,Black,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,NON-TRAUMATIC SEIZURE DISORDER,NaN,NaN,841 Williams Street #1D,CALUMET CITY,-87.545633,41.599656,77889,"(41.599656, -87.5456329)"
77,ME2018-02047,05/04/2018 01:03:00 PM,05/04/2018 01:13:00 PM,77.0,Female,White,False,NATURAL,ORGANIC CARDIOVASCULAR DISEASE,NaN,NaN,NaN,NaN,NaN,1127 DARTMOUTH RD,FLOSSMOOR,-87.665969,41.541767,78304,"(41.5417673, -87.6659691)"


### What are the different types?

In [33]:
df[(df.Primary_Cause.str.contains("CARDIOVASCULAR DISEASE", na=False))].groupby(by='Primary_Cause').Primary_Cause.count().sort_values(ascending=False).head(10)

Primary_Cause
ORGANIC CARDIOVASCULAR DISEASE                              1983
HYPERTENSIVE CARDIOVASCULAR DISEASE                         1177
ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE                      974
HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE         960
HYPERTENSIVE AND ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE     460
HYPERTENSIVE AND ATHEROSCLEROTIC CARDIOVASCULAR DISEASE      298
ATHEROSCLEROTIC CARDIOVASCULAR DISEASE                       140
HYPERTENSIVE-ARTERIOSCLEROTIC CARDIOVASCULAR DISEASE          90
ORGANIC  CARDIOVASCULAR DISEASE                               77
HYPERTENSIVE ATHEROSCLEROTIC CARDIOVASCULAR DISEASE           38
Name: Primary_Cause, dtype: int64

### Replace all of those with a nice simple 'CARDIOVASCULAR DISEASE'

- *Tip: you can use `.replace` or `.str.replace`, but they each involve different things! I suggest `.replace`, it looks a little cleaner in this situation*
- *Tip: for `.replace`, you need to give it more options than usual*
- *Tip: for `.str.replace`, it won't automatically save back into the column, you need to do that yourself*

In [34]:
df['Primary_Cause'] = df.Primary_Cause.str.replace('.*CARDIOVASCULAR D.*', 'CARDIOVASCULAR DISEASE')

### Check the top 5 primary causes. Cardiovascular disease should be first with about 28.4%

In [35]:
df.Primary_Cause.value_counts(normalize=True).sort_values(ascending=False).head(5)

CARDIOVASCULAR DISEASE     0.286228
MULTIPLE GUNSHOT WOUNDS    0.061173
MULTIPLE INJURIES          0.041804
PENDING                    0.020080
GUNSHOT WOUND OF HEAD      0.019547
Name: Primary_Cause, dtype: float64

We could also clean up gunshots, but... let's just move on.

# The Opioid Epidemic

America has a [big problem with fentanyl](https://www.theatlantic.com/health/archive/2018/05/americas-opioid-crisis-is-now-a-fentanyl-crisis/559445/) and other opioids.

## Find all of the rows where fentanyl was part of the primary cause of death

We don't need `na=False` any more because we *dropped the rows without primary causes*.

In [36]:
df[(df.Primary_Cause.str.contains("FENTANYL"))]


,Case_Number,Date_of_Incident,Date_of_Death,Age,Gender,Race,Latino,Manner_of_Death,Primary_Cause,Primary_Cause_Line_B,Primary_Cause_Line_C,Secondary_Cause,Gun_Related,Opioid_Related,Incident_Address,Incident_City,longitude,latitude,OBJECTID,location
15,ME2018-01127,03/03/2018 02:15:00 AM,03/03/2018 02:24:00 AM,37.0,Male,White,False,ACCIDENT,"COMBINED DRUG (COCAINE, HEROIN, FENTANYL) TOXI...",NaN,NaN,NaN,NaN,True,6547 W. 111TH APT W-27,WORTH,-87.781732,41.689640,63784,"(41.6896403, -87.7817317)"
53,ME2018-00643,02/03/2018 10:45:00 AM,02/03/2018 11:11:00 AM,35.0,Male,White,False,ACCIDENT,"ETHANOL AND COMBINED DRUG (PROBABLE HEROIN, CY...",NaN,NaN,NaN,NaN,True,"2540 N. Manheim Road, Rm#215",FRANKLIN PARK,NaN,NaN,77879,NaN
54,ME2018-00671,02/05/2018 12:00:00 AM,02/05/2018 12:10:00 AM,27.0,Male,White,False,ACCIDENT,COMBINED FENTANYL AND PROBABLE HEROIN TOXICITY,NaN,NaN,OBESITY,NaN,True,3435 W. VAN BUREN,CHICAGO,-87.711994,41.875391,77910,"(41.8753913, -87.7119935)"
69,ME2018-01356,03/15/2018 01:31:00 PM,03/15/2018 01:44:00 PM,26.0,Male,White,False,ACCIDENT,"FENTANYL, HEROIN, AND ALPRAZOLAM TOXICITY",NaN,NaN,NaN,NaN,True,710 N. 4TH AVENUE,MAYWOOD,-87.838564,41.893982,77870,"(41.8939818, -87.8385636)"
93,ME2018-01560,03/29/2018 06:50:00 PM,03/29/2018 07:12:00 PM,38.0,Male,Asian,False,ACCIDENT,COCAINE AND FENTANYL TOXICITY,NaN,NaN,NaN,NaN,True,535 N MICHIGAN AVE #2705,CHICAGO,-87.623541,41.891975,66297,"(41.8919752, -87.6235405)"
99,ME2018-01570,03/30/2018 03:33:00 PM,03/30/2018 04:20:00 PM,50.0,Male,White,False,ACCIDENT,"COCAINE, FENTANYL AND ACETYL FENTANYL TOXICITY",NaN,NaN,NaN,NaN,True,1042 N. KNOLLWOOD DR. UNIT A,SCHAUMBURG,-88.138219,42.044579,62417,"(42.0445787, -88.1382195)"
102,ME2018-01574,03/30/2018 10:43:00 PM,03/30/2018 11:03:00 PM,40.0,Female,White,False,ACCIDENT,"COMBINED DRUG (FENTANYL, METHADONE, AND ALPRAZ...",NaN,NaN,NaN,NaN,True,4345 AUTHUR AVE.,BROOKFIELD,NaN,NaN,63618,NaN
106,ME2018-01103,03/01/2018 02:02:00 PM,03/01/2018 02:45:00 PM,68.0,Male,Black,False,ACCIDENT,COMBINED DRUG (FENTANYL AND METHADONE) TOXICITY,NaN,NaN,NaN,NaN,True,1541 SOUTH HAMLIN,CHICAGO,-87.719819,41.859519,78255,"(41.8595195, -87.7198186)"
107,ME2018-02197,05/07/2018 09:27:00 AM,05/08/2018 05:23:00 AM,38.0,Female,White,False,ACCIDENT,FENTANYL AND COCAINE TOXICITY,NaN,NaN,NaN,NaN,True,15611 DIXIE HWY (APT 2),HARVEY,-87.669333,41.605710,78760,"(41.6057103, -87.6693327)"
109,ME2017-05643,12/02/2017 02:50:00 PM,12/02/2017 02:44:00 PM,45.0,Male,White,False,ACCIDENT,"COMBINED DRUG (HEROIN, FENTANYL, AND ACETYL FE...",NaN,NaN,NaN,NaN,True,4023 West Congress Parkway,CHICAGO,-87.726373,41.874299,77943,"(41.8742991, -87.7263732)"


## Fentanyl and race

In the late 80's and 90's, the [crack cocaine epidemic](https://en.wikipedia.org/wiki/Crack_epidemic) swept through inner cities in the US. It was treated primarily as a crime problem, while many people say fentanyl and heroin overdoses are being treated as a medical problem due to the racial differences - the crack epidemic mainly affected Black communities, while fentanyl seems to be a problem for everyone.

### How does the racial breakdown of fentanyl deaths compare to the racial breakdown of other causes of death? How about compared to causes of accidental death?

In [37]:
df[(df.Primary_Cause.str.contains("FENTANYL"))].Race.value_counts()

White              830
Black              650
Asian                6
Other                5
American Indian      1
Unknown              1
Name: Race, dtype: int64

In [38]:
df[(df.Manner_of_Death.str.contains("ACCIDENT", na=False))].Race.value_counts()

White              4885
Black              2508
Other               106
Asian               103
Unknown               7
American Indian       7
Name: Race, dtype: int64

### Now compare it to homicides

In [39]:
df[(df.Manner_of_Death.str.contains("HOMICIDE", na=False))].Race.value_counts()

Black      2195
White       601
Other        33
Asian         5
Unknown       1
Name: Race, dtype: int64

### Now compare it to suicide

In [40]:
df[(df.Manner_of_Death.str.contains("SUICIDE", na=False))].Race.value_counts()

White      1366
Black       263
Asian        60
Other        21
Unknown       1
Name: Race, dtype: int64

## These differences seems kind of crazy

Let's look at all of these at once: I want a breakdown of the most common manners of death for **men**, based on race.

Percentages, please, not raw numbers.

You can look at women, too, although I think the numbers are more surprising for men.

In [89]:
df[(df.Gender == 'Male') & (df.Race == 'White')].Manner_of_Death.count()

8816

In [90]:
df[(df.Gender == 'Male') & (df.Race == 'Black')].Manner_of_Death.count()

6753

In [91]:
df[(df.Gender == 'Male') & (df.Race == 'Asian')].Manner_of_Death.count()

229

In [92]:
df[(df.Gender == 'Male') & (df.Race == 'American Indian')].Manner_of_Death.count()

10

## Back to drugs: what is the most popular opioid-related primary cause of death that does NOT involve fentanyl?

- *Tip: Pay attention to your column names! There's one that might tell you if something is opioid-related...*
- *Tip: Usually you can use `not` or `!` to means "not", but for pandas and `.isin` or `.str.contains` you need to use `~`*
- *Tip: For "and" in pandas you'll need to use `&`, and make sure all of your clauses have parens around them, e.g. `df[(df.col1 = 'A') & (df.col2 = 'B')]`.*

In [109]:
df[(~df.Opioid_Related.isna()) & (~df.Primary_Cause.str.contains("FENTANYL"))].Primary_Cause.value_counts()

HEROIN TOXICITY                                                                                                             286
HEROIN INTOXICATION                                                                                                          61
ACUTE HEROIN TOXICITY                                                                                                        59
HEROIN AND ETHANOL TOXICITY                                                                                                  37
COMBINED HEROIN AND ETHANOL TOXICITY                                                                                         34
OPIATE TOXICITY                                                                                                              26
COCAINE AND HEROIN TOXICITY                                                                                                  25
CARDIOVASCULAR DISEASE                                                                                  

# How do heroin and fentanyl deaths compare?

## Count the number of deaths involving heroin, the number of deaths involving fentanyl, and the number of deaths involving both.

- *Tip: This will take 3 different statements*
- *Tip: You should get `813` that include both*

## That's weird.

I heard fentanyl really surpassed heroin in the past few years. Let's see how this 

### Pull the year out and store it in a new column called `year`

If you run `df['Date of Incident'].str.extract("(\d\d\d\d)", expand=False)`, it will pull out the year of each incident. **Store this in a new column called `year`.**

(It's regular expression stuff. `\d\d\d\d` means "four numbers in a row", and `()` + `.str.extract` means "pull it out".)

### What is the datatype of the new `year` column?

## Convert this new column to an integer and save it back on top of itself

- *Tip: This uses is your friend `.astype`*
- *Tip: Make sure to save it back on top of itself!*

## Confirm the column is a number

## Plot the number of opioid deaths by year

If you'd like to make it look nicer, do some sorting and get rid of 2018.

- *Tip: Think of it in a few steps. First, filter for opioid deaths. Then get the number of deaths for each year. Then plot it.*
- *Tip: What's up with 2018? Why's it look so weird? Can you get rid of it? Remember to use lots of parens!*
- *Tip: Make sure the earliest year is on the left. You might need to sort by something other than values.*

## Plot the number of fentanyl deaths by year, and the number of heroin deaths by year

- *Tip: You'll want to look up how to use `ylim` - it will let you set each graphic to use the same scale. This should be separate graphics.*
- *Tip: Pay attention to the numbers on your axes. `sort_index()` will be your friend.*
- *Tip: You should probably get rid of 2018*

## How does this compare to gun deaths?

## But hey: numbers can lie pretty easily!

The numbers are just so low in 2014 and much higher in 2017. What's going on there?

Well, maybe **there just isn't as much data from the earlier years**. Plot how many entries there are for each year.

And we don't know the best way to fix that up yet, so instead I'm going to give you a present.

# Is the true lesson here, don't move to Cook County, Illinois?

Cook County is basically Chicago. It's probably just certain areas that are trouble, right? Let's investigate that without even having a clue how mapping works.

## Fun bonus: Making cheating maps

### Make a new dataframe of every death in the actual city of Chicago

### Confirm this new dataframe has 13,627 rows

### Use lat and long in the worst way possible to make a map

Use `longitude` and `latitude` and `plot` to make a rough map of the city. Chicago [looks like this](https://en.wikipedia.org/wiki/File:DuPage_County_Illinois_Incorporated_and_Unincorporated_areas_Chicago_Highlighted.svg)

- *Tip: Use the `latitude` and `longitude` columns*
- *Tip: You don't want a line graph, of course. Or a bar. What kind is the kind with dots on it?*
- *Tip: Use something like like `figsize=(10,5)` to specify the height and width of the map (but, you know, with better numbers that make it look like chicago)*

## Now let's find out where to live

Make a map of every non-homicide death in Chicago, then plot the homicides on top of it.

Use the `ax=df.plot` trick from the beer cans assignment to plot all of the rows representing homicides vs non-homicides. You can use `color='red'` to make one of them red, and `alpha=0.05` to make each mark very transparent to allow them to layer on top of each other.

## Never tell anyone I let you do that.

But you want to see something actually completely legitimately insane?

**Chicago is one of the most segregated cities in America.** If you'd like to see this for yourself, make a map of `Race`. Plot black vs white in a way similar to what we did above.

Yup.